In [ ]:
stack = [3, 4, 5]
stack.append(6)
stack.append(7)
stack

In [ ]:
stack.pop()

In [ ]:
stack

In [ ]:
stack.pop()

In [ ]:
stack.pop()

In [ ]:
stack

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

In [ ]:
plt.style.use('ggplot')

In [3]:
df = pd.read_csv('djia.csv', comment='#', parse_dates=[0], index_col=0,names=['date', 'djia'])

In [4]:
df

,djia
date,
1885-02-16,30.9226
1885-02-17,31.3365
1885-02-18,31.4744
1885-02-19,31.6765
1885-02-20,31.4252
...,...
2024-02-23,39131.5300
2024-02-26,39069.2300
2024-02-27,38972.4100


In [ ]:
plt=df.plot(figsize=(12,6))
plt.fmt_xdata = mdates.DateFormatter('%Y-%m-%d')
min_x = df.index.min()
max_x = df.index.max()
plt.set_xlim([min_x,max_x])
ticks = pd.date_range(start=min_x,end =max_x, freq = '10A')
ticks = ticks.append(pd.DatetimeIndex(['2024-02-29']))
_=plt.set_xticks(ticks)

In [ ]:
def simple_stock_span(quotes: list[float]) -> list[int]:
    spans = []
    for i in range(len(quotes)):
        k = 1
        span_end = False
        while i - k >= 0 and not span_end:
            if quotes[i - k] <= quotes[i]:
                k += 1
            else:
                span_end = True
        spans.append(k)
    return spans

In [ ]:
def read_quotes(filename: str) -> tuple[list[str],list[float]]:
    dates=[]
    quotes =[]
    with open(filename) as quotes_file:
        for line in quotes_file:
            if line.startswith("#"):
                continue
            parts = line.split(',')
            if len(parts) !=2:
                continue
            month, day, year = parts[0].split('/')
            date = '/'.join((year, month, day))
            dates.append(date)
            quotes.append(float(parts[-1]))
    return dates, quotes

In [ ]:
df.columns

In [5]:
dates = df.index

In [6]:
dates

DatetimeIndex(['1885-02-16', '1885-02-17', '1885-02-18', '1885-02-19',
               '1885-02-20', '1885-02-21', '1885-02-24', '1885-02-25',
               '1885-02-26', '1885-02-27',
               ...
               '2024-02-15', '2024-02-16', '2024-02-20', '2024-02-21',
               '2024-02-22', '2024-02-23', '2024-02-26', '2024-02-27',
               '2024-02-28', '2024-02-29'],
              dtype='datetime64[ns]', name='date', length=37963, freq=None)

In [7]:
quotes =list(df['djia'])

In [8]:
quotes

[30.9226,
 31.3365,
 31.4744,
 31.6765,
 31.4252,
 31.8933,
 32.3318,
 32.8491,
 32.3318,
 31.7651,
 31.8982,
 31.9425,
 31.8292,
 31.568,
 31.7799,
 31.7109,
 32.1987,
 32.2874,
 32.2923,
 32.514,
 32.2184,
 31.9819,
 32.1297,
 31.9868,
 31.6518,
 31.4695,
 31.0753,
 31.0014,
 30.5875,
 30.829,
 30.4841,
 30.7403,
 30.9965,
 31.0063,
 30.962,
 30.6023,
 30.7107,
 30.7452,
 30.7994,
 30.7748,
 30.7452,
 30.3461,
 30.4545,
 31.2133,
 30.8832,
 31.4054,
 31.3857,
 31.5188,
 32.0854,
 31.7897,
 31.5779,
 31.3857,
 31.5779,
 31.499,
 31.5927,
 31.6321,
 31.4843,
 31.4498,
 31.5188,
 31.5927,
 31.6321,
 31.7306,
 31.5188,
 31.435,
 30.6614,
 30.5087,
 30.824,
 30.5038,
 30.9226,
 31.1443,
 30.7945,
 30.6565,
 30.6565,
 30.6565,
 30.8733,
 31.031,
 31.095,
 31.1,
 30.9423,
 31.1246,
 31.3611,
 31.4843,
 31.0655,
 31.1936,
 31.026,
 30.9176,
 30.6861,
 30.5383,
 30.7156,
 30.9078,
 30.7107,
 30.4693,
 30.5481,
 30.4151,
 30.5136,
 30.5875,
 30.8634,
 31.0211,
 31.0507,
 31.2921,
 31.1837,
 31

In [ ]:
spans_simple = simple_stock_span(quotes)

In [ ]:
spans_simple[-10:]

In [ ]:
max_value=max(spans_simple)

In [ ]:
max_value

In [ ]:
max_value_index =spans_simple.index(max_value)

In [ ]:
max_value_index

In [ ]:
df.index[max_value_index]

In [ ]:
import time

iterations =10
start =time.time()
for i in range(iterations):
    simple_stock_span(quotes)
end = time.time()

time_simple =(end-start)/iterations
time_simple

In [16]:
def stack_stock_span(quotes: list[float]) -> list[int]:
    spans =[1]
    s=[]
    s.append(0)
    for i in range(1,len(quotes)):
        while len(s)!= 0 and quotes[s[-1]] <=quotes[i]:
            s.pop()
        if len(s) ==0:
            spans.append(i+1)
        else:
            spans.append(i-s[-1])
        s.append(i)
    return spans

In [10]:
class StockSpanner:
    def __init__(self):
        self.stack =[] #pair:(price,span)
        
    def next(self, price:float)->int:
        span =1
        while self.stack and self.stack[-1][0] <=price:
            span+= self.stack[-1][1]
            self.stack.pop()
        self.stack.append([price,span])
        
        return self.stack[-1][1]

In [13]:
spanner = StockSpanner()

In [14]:
spans = [spanner.next(price) for price in quotes ]

In [18]:
import time

iterations =10
start1 =time.time()
for i in range(iterations):
    stack_stock_span(quotes)
end1 = time.time()

start2=time.time()
for i in range(iterations):
    spans = [spanner.next(price) for price in quotes ]
end2 = time.time()

time_span =(end1-start1)/iterations
time_spanner =(end2-start2)/iterations

print(f"time_span:{time_span},spanner:{time_spanner}")

time_span:0.012392830848693848,spanner:0.01716177463531494
